In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
information on loader: https://tenthousandmeters.com/blog/python-behind-the-scenes-11-how-the-python-import-system-works/

Problem: The loaders don't always point to the same object for the same module
but imported in different modules
"""

"\ninformation on loader: https://tenthousandmeters.com/blog/python-behind-the-scenes-11-how-the-python-import-system-works/\n\nProblem: The loaders don't always point to the same object for the same module\nbut imported in different modules\n"

In [3]:
from python_tools import ipyvolume_utils as ipvu

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Exploring class

In [135]:
import abc
import uuid


class Grader:
    def __init__(self):
        self.student_graders = {}
        self.assignment_classes = {}

    def register(self, assignment_class):
        if not issubclass(assignment_class, Assignment):
            raise RuntimeError(
                "Your class does not have the right methods"
            )

        id = uuid.uuid4()
        self.assignment_classes[id] = assignment_class
        return id

    def start_assignment(self, student, id):
        self.student_graders[student] = AssignmentGrader(
            student, self.assignment_classes[id]
        )

    def get_lesson(self, student):
        assignment = self.student_graders[student]
        return assignment.lesson()

    def check_assignment(self, student, code):
        assignment = self.student_graders[student]
        return assignment.check(code)

    def assignment_summary(self, student):
        grader = self.student_graders[student]
        return f"""
        {student}'s attempts at {grader.assignment.__class__.__name__}:

        attempts: {grader.attempts}
        correct: {grader.correct_attempts}

        passed: {grader.correct_attempts > 0}
        """


class Assignment(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def lesson(self, student):
        pass

    @abc.abstractmethod
    def check(self, code):
        pass

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Assignment:
            attrs = set(dir(C))
            if set(cls.__abstractmethods__) <= attrs:
                return True

        return NotImplemented


class AssignmentGrader:
    def __init__(self, student, AssignmentClass):
        self.assignment = AssignmentClass()
        self.assignment.student = student
        self.attempts = 0
        self.correct_attempts = 0

    def check(self, code):
        self.attempts += 1
        result = self.assignment.check(code)
        if result:
            self.correct_attempts += 1

        return result

    def lesson(self):
        return self.assignment.lesson()
    
class IntroToPython:
    def lesson(self):
        return f"""
            Hello {self.student}. define two variables,
            an integer named a with value 1
            and a string named b with value 'hello'

        """

    def check(self, code):
        return code == "a = 1\nb = 'hello'"


class Statistics(Assignment):
    def lesson(self):
        return (
            "Good work so far, "
            + self.student
            + ". Now calculate the average of the numbers "
            + " 1, 5, 18, -3 and assign to a variable named 'avg'"
        )

    def check(self, code):
        import statistics

        code = "import statistics\n" + code

        local_vars = {}
        global_vars = {}
        exec(code, global_vars, local_vars)

        return local_vars.get("avg") == statistics.mean([1, 5, 18, -3])


# Some code to test that it's doing the right thing

In [137]:
new_obj = Statistics()
new_obj.lesson()

AttributeError: 'Statistics' object has no attribute 'student'

In [131]:
class PrintClass:
    def print_name(self):
        print(f"{self.name}")
    def print_attr_1(self):
        print(f"{self.attr_1}")
    def print_attr_2(self):
        print(f"{self.attr_2}")
    def print_attr_base(self):
        print(f"{self.attr_base}")

class Comp2(PrintClass):
    attr_2 = 30

class Comp1(PrintClass):
    attr_1 = 20
    def __init__(self):
        self.component = Comp2()

class MyBase(PrintClass):
    attr_base = 10
    def __init__(self):
        self.name = "Brendan"
        self.component_big = Comp1()
        

        

In [133]:
b = MyBase()
b.component_big.component.print_attr_1()

AttributeError: 'Comp2' object has no attribute 'attr_1'

# List of derived_products

In [ ]:
"""
How we could devide it?
- each stage have it's own dataclass that 
  is an objedct of another datacalss
    

"""

In [ ]:
"""
decimation: 
- time
- Original mesh?

Soma Identification:
- time
- nuclei_pieces
- glia_pieces
- total_soma_list
- total_soma_list_sdf

Decomposition
- time

Splitting Preview:
- time
- split_results
- red_blue_split_results

Splitting:
- time
- neuron_list_errored_limbs_area
- neuron_list_errored_limbs_skeletal_length
- neuron_list_n_multi_soma_errors
- neuron_list_n_same_soma_errors

Cell Typing:
- time (could have subcategories for like spine, synapse)
- filtering_info_low_branch

- winning_nucleus_id
- nucleus_info

- database_cell_type_info
- database_e_i_class

- baylor_e_i
- baylor_cell_type_info

- e_i_class

# axon dendrite
filtering_info
axon_angles_dict
G_axon_labeled

Proofreading
"""

In [119]:
from neurd.products import StageProducts,PipelineProducts

pp = StageProducts(cell_type="excitatory")
p = PipelineProducts(
    cell_type = pp
)

p.export()

{'cell_type': {'cell_type': 'excitatory'}}

In [121]:
p.set_stage_products(
    "new_stage",
    hello = 10,
    hi = 5
)

In [125]:
p.products["new_stage"].export()

{'hello': 10, 'hi': 5}

In [115]:
PipelineProducts(p).export()

{'cell_type': {'cell_type': 'excitatory'}}

['cell_type']

# Seeing if can automatically close the legend

In [4]:
"""
1) Have these vuetify ui components (shown below) ipyvolume > vue > container.vue

2) Using a veutify template to make the component: 
https://ipyvuetify.readthedocs.io/en/latest/template_usage.html

<v-expansion-panel>
    <v-expansion-panel-header>Legend</v-expansion-panel-header>
    <v-expansion-panel-content>
        <jupyter-widget ref="legend" :widget="legend"></jupyter-widget>
    </v-expansion-panel-content>
</v-expansion-panel>

2) Use a python module for manipulating the vuetify templates (ipyvuetify package)

Notes ---
1) a vue object could be shown or hidden using built in methods: 
- hide(), show()
2) some vue objects can be accessed with children 

w = v.Icon(children=['mdi-eye-off'])
w.hide()
w

"""

'\n1) Have these vuetify ui components (shown below) ipyvolume > vue > container.vue\n\n2) Using a veutify template to make the component: \nhttps://ipyvuetify.readthedocs.io/en/latest/template_usage.html\n\n<v-expansion-panel>\n    <v-expansion-panel-header>Legend</v-expansion-panel-header>\n    <v-expansion-panel-content>\n        <jupyter-widget ref="legend" :widget="legend"></jupyter-widget>\n    </v-expansion-panel-content>\n</v-expansion-panel>\n\n2) Use a python module for manipulating the vuetify templates (ipyvuetify package)\n\nNotes ---\n1) a vue object could be shown or hidden using built in methods: \n- hide(), show()\n2) some vue objects can be accessed with children \n\nw = v.Icon(children=[\'mdi-eye-off\'])\nw.hide()\nw\n\n'

In [32]:
from mesh_tools import trimesh_utils as tu
import numpy as np
mesh = tu.sphere_mesh()
mesh

<trimesh.Trimesh(vertices.shape=(42, 3), faces.shape=(80, 3))>

In [34]:
from python_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(mesh)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

In [ ]:
container.close_all()

In [ ]:
import ipyvuetify as v

In [25]:
import ipyvuetify as v
container.traits()

{'_component_instances': <traitlets.traitlets.List at 0x7f82224bcee0>,
 '_dom_classes': <ipywidgets.widgets.trait_types.TypedTuple at 0x7f8232e65c40>,
 '_jupyter_vue': <traitlets.traitlets.Any at 0x7f82224bcb80>,
 '_model_module': <traitlets.traitlets.Unicode at 0x7f822211a520>,
 '_model_module_version': <traitlets.traitlets.Unicode at 0x7f822211adc0>,
 '_model_name': <traitlets.traitlets.Unicode at 0x7f822211aa60>,
 '_msg_callbacks': <traitlets.traitlets.Instance at 0x7f8232e65730>,
 '_property_lock': <traitlets.traitlets.Dict at 0x7f8232e656d0>,
 '_states_to_send': <traitlets.traitlets.Set at 0x7f8232e65700>,
 '_view_count': <traitlets.traitlets.Int at 0x7f8232e655e0>,
 '_view_module': <traitlets.traitlets.Unicode at 0x7f822211a5e0>,
 '_view_module_version': <traitlets.traitlets.Unicode at 0x7f822211ad60>,
 '_view_name': <traitlets.traitlets.Unicode at 0x7f822211aaf0>,
 'children': <traitlets.traitlets.List at 0x7f822249e790>,
 'comm': <traitlets.traitlets.Any at 0x7f8232e65610>,
 'c

In [ ]:
type(container.legend)

In [ ]:
import ipywidgets as widgets
widgets.widget_serialization?

In [ ]:
import ipyvolume as ipv
[k for k in dir(container) if "legend" in k.lower()]

In [ ]:
container.legend

In [ ]:
dir(container)

In [ ]:
type(container.legend)

In [ ]:
from neurd import neuron_visualizations as nviz
from neurd import preprocessing_vp2 as pre

In [ ]:
from neurd import preprocessing_vp2 as pre
pre.nviz.nviz

In [ ]:
from neurd.microns_volume_utils import data_interface as hdju

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.nviz

In [ ]:
from neurd import synapse_utils as syu
syu.hdju

In [ ]:
hdju

In [ ]:
hdju.set_synapse_filepath("hello")

In [ ]:
syu.hdju

In [ ]:
from importlib import reload
from neurd import classification_utils as clu
clu = reload(clu)

In [ ]:
from python_tools import package_utils as pku
pkg_mods = pku.submodule_version_dict(
    "nviz",
    "neurd",
    use_loader=True,
    verbose = True
)
pkg_mods

In [ ]:
from neurd import neuron_visualizations as nviz
from os import sys
nviz.__loader__
sys.modules["neurd.neuron_visualizations"]

In [ ]:
pre.nviz

In [ ]:
import neurd

neurd.set_volume_params("microns")

pkg_mods = pku.submodule_version_dict(
    "hdju",
    "neurd",
    use_loader=False,
    verbose = True,
)
pkg_mods

In [ ]:
from neurd import axon_utils as au
nviz.au.__loader__ is au.__loader__

In [ ]:
import importlib
from os import sys
def reload_all_modules_in_package(package_name):
    """
    Purpose: To reload all modules in a specified package

    """
    for module in [k for k in sys.modules.values() if f"'{package_name}." in str(k)]:
        importlib.reload(module)

In [ ]:
import os
os.listdir(os.path.dirname("./"))

In [ ]:
nviz.nviz

In [ ]:
nviz.soma_color

In [ ]:
from neurd import neuron_utils as nru
nru.__loader__

In [ ]:
from importlib import reload
pre = reload(pre)

In [ ]:
import ipywidgets
ipywidgets.__version__

In [ ]:
import numpy as np
ipvu.plot_scatter(
    array=np.array([[0.12068656, 0.34903741, 0.84620654],
       [0.04812803, 0.14411268, 0.54990798],
       [0.79566505, 0.63533888, 0.03276685],
       [0.55378168, 0.25424957, 0.71898828],
       [0.1054361 , 0.70734158, 0.25456412],
       [0.51331846, 0.46793084, 0.44032635],
       [0.58216843, 0.46214255, 0.12553144],
       [0.08834693, 0.13477684, 0.88986768],
       [0.39226641, 0.96131601, 0.05423914],
       [0.86158737, 0.30404951, 0.42807901]]),
    color=(1,0,0,0.99),
    size=10,
    widgets_to_plot = ("size","marker","color"),
)

# Look at how proximities are

In [ ]:
from os import sys
sys.path.append("/neurd_packages/meshAfterParty/meshAfterParty/")

In [ ]:
from dataInterfaceMinnie65 import data_interface as hdju

In [ ]:
from neurd import neuron_visualizations as nviz
from python_tools import ipyvolume_utils as ipvu
import numpy as np
ipvu.plot_objects(
    scatters=[np.array([[0.12068656, 0.34903741, 0.84620654],
       [0.04812803, 0.14411268, 0.54990798],
       [0.79566505, 0.63533888, 0.03276685],
       [0.55378168, 0.25424957, 0.71898828],
       [0.1054361 , 0.70734158, 0.25456412],
       [0.51331846, 0.46793084, 0.44032635],
       [0.58216843, 0.46214255, 0.12553144],
       [0.08834693, 0.13477684, 0.88986768],
       [0.39226641, 0.96131601, 0.05423914],
       [0.86158737, 0.30404951, 0.42807901]])],
    scatters_colors=(1,0,0,0.99),
    scatter_size=10,
    #widgets_to_plot = ("size","marker","color"),
)

In [ ]:
from python_tools import ipyvolume_utils as ipvu
import numpy as np
ipvu.plot_objects(
    scatters=[np.random.rand(10,3)],
    scatter_size=10,
    #scatters_colors="red",
    set_zoom=1,
    axis_box_off=False,
    
)

In [ ]:
hdju.plot_proximity(
    864691134884741626,864691134884760314
)

In [ ]:
hdju.plot_proofread_neuron(864691134884741626)

In [ ]:
import neurd

neurd.set_volume_params(
    "microns"
)

hdju.plot_proofread_neuron(864691134884760314)

In [ ]:
hdju.proximity_table & dict(segment_id_post=864691134884760314)